# Analyze Results of a Hyperparameter Tuning job

Once you have completed a tuning job, (or even while the job is still running) you can use this notebook to analyze the results to understand how each hyperparameter effects the quality of the model.

---
## Set up the environment
To start the analysis, you must pick the name of the hyperparameter tuning job.

In [2]:
import boto3
import sagemaker
import os

region = boto3.Session().region_name
sage_client = boto3.Session().client("sagemaker")

## You must have already run a hyperparameter tuning job to analyze it here.
## The Hyperparameter tuning jobs you have run are listed in the Training section on your SageMaker dashboard.
## Copy the name of a completed job you want to analyze from that list.
## For example: tuning_job_name = 'mxnet-training-201007-0054'.
tuning_job_name = "xgboost-211106-0216"

## Track hyperparameter tuning job progress
After you launch a tuning job, you can see its progress by calling describe_tuning_job API. The output from describe-tuning-job is a JSON object that contains information about the current state of the tuning job. You can call list_training_jobs_for_tuning_job to see a detailed list of the training jobs that the tuning job launched.

In [3]:
# run this cell to check current status of hyperparameter tuning job
tuning_job_result = sage_client.describe_hyper_parameter_tuning_job(
    HyperParameterTuningJobName=tuning_job_name
)

status = tuning_job_result["HyperParameterTuningJobStatus"]
if status != "Completed":
    print("Reminder: the tuning job has not been completed.")

job_count = tuning_job_result["TrainingJobStatusCounters"]["Completed"]
print("%d training jobs have completed" % job_count)

objective = tuning_job_result["HyperParameterTuningJobConfig"]["HyperParameterTuningJobObjective"]
is_minimize = objective["Type"] != "Maximize"
objective_name = objective["MetricName"]

20 training jobs have completed


In [4]:
from pprint import pprint

if tuning_job_result.get("BestTrainingJob", None):
    print("Best model found so far:")
    pprint(tuning_job_result["BestTrainingJob"])
else:
    print("No training jobs have reported results yet.")

Best model found so far:
{'CreationTime': datetime.datetime(2021, 11, 6, 2, 43, 39, tzinfo=tzlocal()),
 'FinalHyperParameterTuningJobObjectiveMetric': {'MetricName': 'validation:auc',
                                                 'Value': 0.7779889702796936},
 'ObjectiveStatus': 'Succeeded',
 'TrainingEndTime': datetime.datetime(2021, 11, 6, 2, 48, 1, tzinfo=tzlocal()),
 'TrainingJobArn': 'arn:aws:sagemaker:us-east-1:875692608981:training-job/xgboost-211106-0216-020-7002c02b',
 'TrainingJobName': 'xgboost-211106-0216-020-7002c02b',
 'TrainingJobStatus': 'Completed',
 'TrainingStartTime': datetime.datetime(2021, 11, 6, 2, 46, 39, tzinfo=tzlocal()),
 'TunedHyperParameters': {'alpha': '1.9463703701427026',
                          'eta': '0.08284496023966385',
                          'max_depth': '6',
                          'min_child_weight': '1.026032821221861'}}


## Fetch all results as DataFrame
We can list hyperparameters and objective metrics of all training jobs and pick up the training job with the best objective metric.

In [5]:
import pandas as pd

tuner = sagemaker.HyperparameterTuningJobAnalytics(tuning_job_name)

full_df = tuner.dataframe()

if len(full_df) > 0:
    df = full_df[full_df["FinalObjectiveValue"] > -float("inf")]
    if len(df) > 0:
        df = df.sort_values("FinalObjectiveValue", ascending=is_minimize)
        print("Number of training jobs with valid objective: %d" % len(df))
        print({"lowest": min(df["FinalObjectiveValue"]), "highest": max(df["FinalObjectiveValue"])})
        pd.set_option("display.max_colwidth", None)  # Don't truncate TrainingJobName
    else:
        print("No training jobs have reported valid results yet.")

df

Number of training jobs with valid objective: 20
{'lowest': 0.7197329998016357, 'highest': 0.7779889702796936}


,alpha,eta,max_depth,min_child_weight,TrainingJobName,TrainingJobStatus,FinalObjectiveValue,TrainingStartTime,TrainingEndTime,TrainingElapsedTimeSeconds
1,1.946370,0.082845,6.0,1.026033,xgboost-211106-0216-020-7002c02b,Completed,0.777989,2021-11-06 02:46:39+00:00,2021-11-06 02:48:01+00:00,82.0
8,1.442342,0.145395,5.0,9.771540,xgboost-211106-0216-013-896b88be,Completed,0.776081,2021-11-06 02:35:53+00:00,2021-11-06 02:36:51+00:00,58.0
9,1.157735,0.106729,3.0,1.342034,xgboost-211106-0216-012-ad6b8352,Completed,0.775955,2021-11-06 02:33:17+00:00,2021-11-06 02:34:10+00:00,53.0
7,1.655638,0.034372,7.0,2.748734,xgboost-211106-0216-014-682b18f7,Completed,0.775562,2021-11-06 02:37:19+00:00,2021-11-06 02:38:30+00:00,71.0
20,1.320902,0.091613,8.0,9.741444,xgboost-211106-0216-001-479b8780,Completed,0.775148,2021-11-06 02:19:23+00:00,2021-11-06 02:20:35+00:00,72.0
10,1.414144,0.172922,2.0,1.662011,xgboost-211106-0216-011-7aa3f306,Completed,0.774060,2021-11-06 02:33:10+00:00,2021-11-06 02:34:17+00:00,67.0
4,1.567869,0.034780,10.0,1.491349,xgboost-211106-0216-017-6ea35f92,Completed,0.773389,2021-11-06 02:41:43+00:00,2021-11-06 02:42:52+00:00,69.0
3,1.587869,0.024780,10.0,1.581349,xgboost-211106-0216-018-12a95a54,Completed,0.772230,2021-11-06 02:41:48+00:00,2021-11-06 02:43:01+00:00,73.0
6,1.943197,0.095163,2.0,2.966263,xgboost-211106-0216-015-805da8cc,Completed,0.771640,2021-11-06 02:37:31+00:00,2021-11-06 02:38:55+00:00,84.0
2,0.045408,0.748159,2.0,7.551407,xgboost-211106-0216-019-62821b51,Completed,0.770781,2021-11-06 02:46:12+00:00,2021-11-06 02:47:50+00:00,98.0


## See TuningJob results vs time
Next we will show how the objective metric changes over time, as the tuning job progresses.  For Bayesian strategy, you should expect to see a general trend towards better results, but this progress will not be steady as the algorithm needs to balance _exploration_ of new areas of parameter space against _exploitation_ of known good areas.  This can give you a sense of whether or not the number of training jobs is sufficient for the complexity of your search space.

In [6]:
import bokeh
import bokeh.io

bokeh.io.output_notebook()
from bokeh.plotting import figure, show
from bokeh.models import HoverTool


class HoverHelper:
    def __init__(self, tuning_analytics):
        self.tuner = tuning_analytics

    def hovertool(self):
        tooltips = [
            ("FinalObjectiveValue", "@FinalObjectiveValue"),
            ("TrainingJobName", "@TrainingJobName"),
        ]
        for k in self.tuner.tuning_ranges.keys():
            tooltips.append((k, "@{%s}" % k))

        ht = HoverTool(tooltips=tooltips)
        return ht

    def tools(self, standard_tools="pan,crosshair,wheel_zoom,zoom_in,zoom_out,undo,reset"):
        return [self.hovertool(), standard_tools]


hover = HoverHelper(tuner)

p = figure(plot_width=900, plot_height=400, tools=hover.tools(), x_axis_type="datetime")
p.circle(source=df, x="TrainingStartTime", y="FinalObjectiveValue")
show(p)

Loading BokehJS ...

## Analyze the correlation between objective metric and individual hyperparameters 
Now you have finished a tuning job, you may want to know the correlation between your objective metric and individual hyperparameters you've selected to tune. Having that insight will help you decide whether it makes sense to adjust search ranges for certain hyperparameters and start another tuning job. For example, if you see a positive trend between objective metric and a numerical hyperparameter, you probably want to set a higher tuning range for that hyperparameter in your next tuning job.

The following cell draws a graph for each hyperparameter to show its correlation with your objective metric.

In [7]:
ranges = tuner.tuning_ranges
figures = []
for hp_name, hp_range in ranges.items():
    categorical_args = {}
    if hp_range.get("Values"):
        # This is marked as categorical.  Check if all options are actually numbers.
        def is_num(x):
            try:
                float(x)
                return 1
            except:
                return 0

        vals = hp_range["Values"]
        if sum([is_num(x) for x in vals]) == len(vals):
            # Bokeh has issues plotting a "categorical" range that's actually numeric, so plot as numeric
            print("Hyperparameter %s is tuned as categorical, but all values are numeric" % hp_name)
        else:
            # Set up extra options for plotting categoricals.  A bit tricky when they're actually numbers.
            categorical_args["x_range"] = vals

    # Now plot it
    p = figure(
        plot_width=500,
        plot_height=500,
        title="Objective vs %s" % hp_name,
        tools=hover.tools(),
        x_axis_label=hp_name,
        y_axis_label=objective_name,
        **categorical_args,
    )
    p.circle(source=df, x=hp_name, y="FinalObjectiveValue")
    figures.append(p)
show(bokeh.layouts.Column(*figures))